In [33]:
import pandas as pd

- **call_df** has telephonic record from 26th May to 27th May
- **lead_df** has lead created data  `Lead Management --> Get lead by Date Range`  from 26th May to 27th May

In [34]:
call_df = pd.read_csv("./data/call_records_2627.csv")
lead_df = pd.read_csv("./data/lsq_2527.csv")

In [35]:
call_df.head()

,EntryId,LeadContactNum,StartTime,EndTime,CallType,Duration,Disposition,_SkillId,_AgentSip,IsAddedByAiSvc,_Param1,QueueType,QueueId
0,d956ae70-9f5e-48cb-ac37-f334a0f9251d,8008746597,1716611429,1716611433,OUTBOUND,22,LeadCallDisposition.NOANSWER,7272.0,15799,False,Autodialer,CallQueueType.GENERICQUEUE,7272
1,52a4b652-f41d-434c-b388-c07fe05ceb97,9355400044,1716611407,1716611438,OUTBOUND,31,LeadCallDisposition.NOANSWER,7272.0,16837,False,Autodialer,CallQueueType.GENERICQUEUE,7272
2,d81b83b7-58aa-4038-b37b-ec44982c3dc8,8008746597,1716611433,1716611443,OUTBOUND,34,LeadCallDisposition.NOANSWER,7272.0,16836,False,Autodialer,CallQueueType.GENERICQUEUE,7272
3,6bc3248a-1e0d-49e9-a369-738d4f8e7cf5,9676091096,1716611426,1716611453,OUTBOUND,49,LeadCallDisposition.ANSWER,NaN,15798,False,autodialerfollowup,CallQueueType.AGENTQUEUE,15798
4,7d99020a-8a89-4bc8-9580-5d480e95a16a,9652196271,1716611457,1716611460,OUTBOUND,15,LeadCallDisposition.NOANSWER,7272.0,15799,False,Autodialer,CallQueueType.GENERICQUEUE,7272


- We have extracted only the email address, mobile , and created time of a lead.

In [36]:
lead_df.head()

,EmailAddress,Phone,CreatedOn
0,maryankitha50954@gmail.com,+91-7989972999,2024-05-27 17:25:23.000
1,vinay976@gmail.com,+91-8919326475,2024-05-27 12:28:29.000
2,appalarajukorada69@gmail.com,+91-9160601108,2024-05-27 11:44:46.000
3,siddipoguprasangi@gmail.com,+91-9182113317,2024-05-27 11:17:19.000
4,gvreddyjillella@gmail.com,+91-9441733913,2024-05-27 11:04:29.000


In [37]:
len(call_df["LeadContactNum"].unique())

1029

In [38]:
len(lead_df["Phone"].unique())

100

In [39]:
len(lead_df), len(call_df)

(100, 3819)

In [40]:
def clean_phone(num:str)->int:
    """
    Description:
        This function get the mobile number of format +91-xxxxxxxxxx and then extract only the actual number.
    Returns:
        Mobile number
    """
    return int(num.split("-")[1])

In [41]:
lead_df["Phone"] = lead_df["Phone"].apply(clean_phone)

- We do inner join to filter records from call_df that matches the fresh leads from lead_df.

In [42]:
df = pd.merge(call_df, lead_df, left_on="LeadContactNum", right_on="Phone", how="inner")

- We consider the Start time from call_df as the LeadContact time.

In [43]:
df["LeadContactedDate"] = pd.to_datetime(df["StartTime"], unit="s")

In [44]:
df["LeadGeneratedDate"] = pd.to_datetime(df["CreatedOn"])

In [45]:
df.columns

Index(['EntryId', 'LeadContactNum', 'StartTime', 'EndTime', 'CallType',
       'Duration', 'Disposition', '_SkillId', '_AgentSip', 'IsAddedByAiSvc',
       '_Param1', 'QueueType', 'QueueId', 'EmailAddress', 'Phone', 'CreatedOn',
       'LeadContactedDate', 'LeadGeneratedDate'],
      dtype='object')

- Proceed with only the necessary columns `["Phone","LeadGeneratedDate", "LeadContactedDate"]`

In [46]:
df1 = df[["Phone","LeadGeneratedDate", "LeadContactedDate"]]

- Sort based on `LeadContactedDate`

In [47]:
df1_sorted = df1.sort_values(by=["Phone", "LeadContactedDate"], ascending=True)

- We then, filter only the first record for each unique mobile number.

In [48]:
df1_first_contact = df1_sorted.groupby("Phone").first().reset_index()

In [49]:
df1_first_contact

,Phone,LeadGeneratedDate,LeadContactedDate
0,6303172479,2024-05-26 12:18:30,2024-05-26 12:16:27
1,7702839986,2024-05-26 13:16:53,2024-05-26 13:18:27
2,7842271433,2024-05-26 13:23:34,2024-05-26 13:25:25
3,8179770890,2024-05-26 13:10:22,2024-05-26 13:12:28
4,8297575002,2024-05-26 13:46:32,2024-05-26 13:47:46
5,8466993241,2024-05-26 12:16:22,2024-05-26 12:17:57
6,8790283833,2024-05-26 14:03:12,2024-05-26 14:04:57
7,8886426215,2024-05-26 12:47:30,2024-05-26 12:48:47
8,8919134502,2024-05-26 12:58:09,2024-05-26 12:59:39
9,9392730826,2024-05-26 13:17:14,2024-05-26 13:17:00


In [61]:
time_difference = df1_first_contact["LeadContactedDate"]-df1_first_contact["LeadGeneratedDate"]
#time_difference = pd.to_timedelta(time_difference)

In [67]:
time_difference.dt.total_seconds()//60
time_difference.dt.total_seconds()%60


0    -3.0
1     1.0
2     1.0
3     2.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9    -1.0
10    2.0
11   -6.0
12    1.0
13    0.0
14   -8.0
15    2.0
16   -1.0
dtype: float64

In [73]:
def to_minutes_seconds(time_difference)->str:
  """
  Description: 
    Convert the time difference into minutes and seconds.
  Returns:
    Minutes and Seconds
  
  """
  res_time = list()
  total_seconds= time_difference.dt.total_seconds()
  for record in total_seconds:
    minutes = record//60  # Extract minutes
    seconds = record%60  # Extract seconds 
    res_time.append(f"{minutes} mins {seconds} secs")

  return res_time

In [75]:
df1_first_contact["LeadResponseTime"] = to_minutes_seconds(time_difference)

In [76]:
df1_first_contact

,Phone,LeadGeneratedDate,LeadContactedDate,LeadResponseTime
0,6303172479,2024-05-26 12:18:30,2024-05-26 12:16:27,-3.0 mins 57.0 secs
1,7702839986,2024-05-26 13:16:53,2024-05-26 13:18:27,1.0 mins 34.0 secs
2,7842271433,2024-05-26 13:23:34,2024-05-26 13:25:25,1.0 mins 51.0 secs
3,8179770890,2024-05-26 13:10:22,2024-05-26 13:12:28,2.0 mins 6.0 secs
4,8297575002,2024-05-26 13:46:32,2024-05-26 13:47:46,1.0 mins 14.0 secs
5,8466993241,2024-05-26 12:16:22,2024-05-26 12:17:57,1.0 mins 35.0 secs
6,8790283833,2024-05-26 14:03:12,2024-05-26 14:04:57,1.0 mins 45.0 secs
7,8886426215,2024-05-26 12:47:30,2024-05-26 12:48:47,1.0 mins 17.0 secs
8,8919134502,2024-05-26 12:58:09,2024-05-26 12:59:39,1.0 mins 30.0 secs
9,9392730826,2024-05-26 13:17:14,2024-05-26 13:17:00,-1.0 mins 46.0 secs
